# RAG Chatbot - Empathy Training Version

## What This Does

This chatbot answers questions about your PDF documents **AND** trains your empathy skills.

**Features:**
- Ask questions, get answers from your PDFs
- Your questions are analyzed for empathy
- Get empathy score after 20 messages

**You Need:**
- PDF files in your Google Drive
- A free Google Gemini API key
- Internet connection

**Cost:** 100% FREE

---

**Need help?** See STUDENT_GUIDE.md

---
## Step 1: Install Required Libraries

This installs all the tools we need:
- `chromadb`: Creates a searchable database of documents
- `gradio`: Creates the chat interface
- `pypdf`: Reads PDF files
- `sentence-transformers`: Converts text to searchable format
- `google-generativeai`: Connects to Google's free AI (Gemini)

In [ ]:
# Install all required packages (including empathy analysis)
!pip install -q chromadb gradio pypdf sentence-transformers google-generativeai vaderSentiment

print("✅ All libraries installed successfully!")
print("📊 Empathy analysis module included!")
print("\nℹ️  Note: You may see dependency warnings about 'opentelemetry' packages.")
print("   These are non-critical and won't affect functionality. You can safely ignore them.")

---
## Step 2: Import Libraries

Load all the tools we just installed.

In [ ]:
import os
import time
import asyncio
import gradio as gr
import google.generativeai as genai
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

---
## Step 3: Mount Google Drive

This connects to your Google Drive so we can access your PDF files.

**You'll need to:**
1. Click the link that appears
2. Choose your Google account
3. Allow access
4. Your Drive will appear in `/content/drive/MyDrive/`

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("📁 Your files are available at: /content/drive/MyDrive/")

---
## Step 4: Settings - CHANGE THESE! ✏️

**What to do:**
1. Get a FREE API key: https://aistudio.google.com/app/apikey
2. Paste it in Cell 8 below
3. Update your PDF paths
4. (Optional) Change the persona

In [ ]:
# ============================================================================
# 1. API KEY - CHANGE THIS! ✏️
# ============================================================================
# Get your free key: https://aistudio.google.com/app/apikey
GEMINI_API_KEY = "YOUR_API_KEY_HERE"  # ← PASTE YOUR KEY HERE

# ============================================================================
# 2. PERSONA - CUSTOMIZE THIS! ✏️
# ============================================================================
PERSONA_NAME = "Your Persona Name"  # ✏️ CHANGE THIS - e.g., "Albert Einstein", "Oprah Winfrey"

# ✏️ CUSTOMIZE THIS: Describe your persona's speaking style and personality
PERSONA_DESCRIPTION = """
Replace this entire section with your persona's description.

Template:
You are [NAME], [brief description/title/role].
You speak in a [adjective] manner, using [characteristic words/phrases].
You emphasize [what they care about] and often [communication patterns].

Instructions for customization:
- Describe HOW they speak (tone, word choice, sentence structure)
- Include specific phrases or words they commonly use
- Mention what topics/themes they emphasize
- Note any unique speaking patterns or habits
- Keep it focused on communication style, not just biographical facts

Example:
"You are Marie Curie, pioneering scientist. You speak precisely and scientifically,
using terms like 'research,' 'experiment,' and 'discovery.' You emphasize evidence-based
reasoning and the importance of persistence in scientific work."
"""

# ============================================================================
# 3. RESPONSE SETTINGS - OPTIONAL
# ============================================================================
TEMPERATURE = 0.7  # Creativity level (0.0 = focused, 1.0 = creative)
MAX_OUTPUT_TOKENS = 500  # Maximum response length (~375 words)
NUM_RETRIEVED_DOCS = 7  # How many document pieces to search

# ============================================================================
# 4. SOURCE CITATIONS - OPTIONAL
# ============================================================================
SHOW_SOURCES = True  # True = show which PDFs were used, False = hide

# ============================================================================
# 5. PDF FILES - CHANGE THIS! ✏️
# ============================================================================
PDF_PATHS = [
    "/content/drive/MyDrive/your_folder/document1.pdf",  # ← CHANGE THESE
    "/content/drive/MyDrive/your_folder/document2.pdf",  # ← TO YOUR PATHS
    "/content/drive/MyDrive/your_folder/document3.pdf",
]

# ============================================================================
# API SETUP (Don't change this)
# ============================================================================
genai.configure(api_key=GEMINI_API_KEY)  # Connect to Google Gemini
model = genai.GenerativeModel('gemini-2.0-flash')  # Use fast, free model

print("✅ Configuration complete!")
print(f"📋 Persona: {PERSONA_NAME}")
print(f"🤖 Model: gemini-2.0-flash")
print(f"📄 Number of PDF files configured: {len(PDF_PATHS)}")
print(f"🌡️  Temperature: {TEMPERATURE} (range: 0.0-1.0)")
print(f"📊 Documents retrieved per query: {NUM_RETRIEVED_DOCS}")
print(f"📚 Show source citations: {'ON ✅' if SHOW_SOURCES else 'OFF'}")

---
## 🧪 API Connection Test (Run this first!)

**IMPORTANT: Run this cell to verify your API key works BEFORE proceeding!**

This simple test will:
1. Make a quick API call to Gemini
2. Verify your API key is working
3. Show you that the API is responding

If this works, we know the API is fine. If it hangs, we know it's an API/network issue.

In [ ]:
print("🧪 Testing API connection...")
print("=" * 60)

try:
    # Simple test prompt
    test_response = model.generate_content(
        "Say 'Hello! API is working!' in a friendly, enthusiastic style.",
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=50,
        ),
    )
    
    print("✅ SUCCESS! API is working!")
    print(f"\nTest Response: {test_response.text}")
    print("\n" + "=" * 60)
    print("✅ You can proceed with the rest of the notebook!")
    
except Exception as e:
    print(f"❌ API TEST FAILED!")
    print(f"Error: {str(e)}")
    print("\n" + "=" * 60)
    print("⚠️  STOP! Fix this issue before proceeding:")
    print("  1. Check your API key is correct")
    print("  2. Check your internet connection")
    print("  3. Visit https://aistudio.google.com/app/apikey to verify your key")
    print("  4. Check API status at https://status.cloud.google.com/")

---
## Step 5: Load and Process PDF Files

This step:
1. Reads all your PDF files
2. Extracts the text
3. Splits the text into smaller chunks (for better searching)

**This may take a minute depending on the size of your PDFs.**

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extract all text from a PDF file."""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"❌ Error reading {pdf_path}: {str(e)}")
        return ""

def chunk_text(text, chunk_size=1000, overlap=200):
    """Split text into small pieces (chunks) for better searching.
    
    chunk_size=1000: Each piece is 1000 characters (~150 words)
    overlap=200: Pieces overlap by 200 characters to keep context
    """
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        
        # Only add non-empty chunks
        if chunk.strip():
            chunks.append(chunk)
        
        start += chunk_size - overlap  # Move forward, but keep some overlap
    
    return chunks

# Process all PDFs
print("📖 Processing PDF files...\n")
all_chunks = []
metadata = []

for idx, pdf_path in enumerate(PDF_PATHS):
    print(f"Processing: {pdf_path}")
    
    # Check if file exists
    if not os.path.exists(pdf_path):
        print(f"⚠️  Warning: File not found - {pdf_path}")
        continue
    
    # Extract text
    text = extract_text_from_pdf(pdf_path)
    
    if text:
        # Split into chunks
        chunks = chunk_text(text)
        all_chunks.extend(chunks)
        
        # Store metadata for each chunk (tracks which file it came from)
        for chunk_idx, chunk in enumerate(chunks):
            metadata.append({
                "source": os.path.basename(pdf_path),
                "chunk_id": chunk_idx,
                "total_chunks": len(chunks)
            })
        
        print(f"  ✅ Extracted {len(chunks)} chunks")
    else:
        print(f"  ⚠️  No text extracted")

print(f"\n✅ Processing complete!")
print(f"📊 Total chunks created: {len(all_chunks)}")

if len(all_chunks) == 0:
    print("\n⚠️  WARNING: No text was extracted from PDFs!")
    print("Please check:")
    print("  1. PDF file paths are correct")
    print("  2. Files exist in your Google Drive")
    print("  3. PDFs are not password-protected or image-only")

---
## Step 6: Create Vector Database (ChromaDB)

This step:
1. Converts all text chunks into "embeddings" (numerical representations)
2. Stores them in ChromaDB for fast searching
3. Allows us to find relevant context when a question is asked

**This may take 1-2 minutes for large document sets.**

In [ ]:
print("🔧 Initializing embedding model...")
# Load a free search model that converts text to numbers for searching
# Model: 'all-MiniLM-L6-v2' - fast, accurate, and free
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded!")

print("\n🗄️  Creating ChromaDB database...")
# Initialize ChromaDB (our vector database for searching)
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,  # Don't send usage data
    allow_reset=True  # Allow database reset if needed
))

# Create or get collection (a collection is like a folder for documents)
try:
    chroma_client.delete_collection("documents")  # Delete old collection if exists
except:
    pass

collection = chroma_client.create_collection(
    name="documents",
    metadata={"description": f"Document collection for {PERSONA_NAME} chatbot"}
)

print("✅ Database created!")

# Add documents to the database
if len(all_chunks) > 0:
    print(f"\n📥 Adding {len(all_chunks)} chunks to database...")
    print("⏳ This may take a moment...")
    
    # Convert text chunks to numbers (embeddings) for searching
    embeddings = embedding_model.encode(all_chunks, show_progress_bar=True)
    
    # Store in ChromaDB with metadata (source file, chunk number)
    collection.add(
        embeddings=embeddings.tolist(),
        documents=all_chunks,
        metadatas=metadata,
        ids=[f"chunk_{i}" for i in range(len(all_chunks))]
    )
    
    print("✅ All chunks added to database!")
    print(f"📊 Database contains {collection.count()} document chunks")
else:
    print("⚠️  No chunks to add to database!")

---
## Step 7: Create RAG Query Function

This is the "brain" of the chatbot. It:
1. Takes a user's question
2. Searches the database for relevant information
3. Sends the question + relevant context to Google Gemini
4. Returns a response in the persona's style

---
## Step 8: Empathy Analysis Module 📊

**This is the empathy tracking system!**

This module analyzes YOUR messages to measure empathy across 5 dimensions:
1. Sentiment/warmth
2. Open question ratio  
3. Emotion word usage
4. Perspective-taking
5. Active listening

After 20 messages, you'll receive a comprehensive empathy report.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import statistics
import datetime

class EmpathyAnalyzer:
    """Analyzes student messages for empathy indicators."""
    
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()  # Tool to measure positive/negative tone
        self.conversation_data = []  # Stores all analyzed messages
        
    def analyze_message(self, message):
        """Analyze single student message for empathy markers."""
        text = message.lower()
        
        # Store analysis results for this message
        analysis = {
            'message': message,
            'timestamp': datetime.datetime.now().isoformat(),
            'sentiment': self.vader.polarity_scores(text)['compound'],  # -1 (negative) to +1 (positive)
            'open_questions': self._count_open_questions(text),  # How many "how/why" questions
            'emotion_words': self._count_emotion_words(text),  # Words like "feel", "sad", "happy"
            'perspective_taking': self._count_perspective_taking(text),  # Phrases like "you feel", "you think"
            'active_listening': self._count_active_listening(text),  # Phrases like "tell me more"
            'word_count': len(message.split())
        }
        
        self.conversation_data.append(analysis)
        return analysis
    
    def _count_open_questions(self, text):
        """Count open-ended question words (what, how, why, etc.)"""
        open_markers = ['what', 'how', 'why', 'tell me', 'describe', 'explain', 'could you']
        return sum(1 for marker in open_markers if marker in text)
    
    def _count_emotion_words(self, text):
        """Count emotion-related words"""
        emotion_dict = ['feel', 'felt', 'emotion', 'happy', 'sad', 'angry', 'afraid', 
                       'worried', 'excited', 'frustrated', 'anxious', 'upset', 'joy', 
                       'nervous', 'scared', 'content', 'pleased', 'disappointed']
        return sum(text.count(word) for word in emotion_dict)
    
    def _count_perspective_taking(self, text):
        """Count phrases that show understanding of other person's viewpoint"""
        perspective_phrases = ['you feel', 'you think', 'you must', 'your perspective',
                              'in your situation', 'you seem', 'you might', 'from your view',
                              'in your shoes', 'you believe', 'you experience']
        return sum(1 for phrase in perspective_phrases if phrase in text)
    
    def _count_active_listening(self, text):
        """Count phrases that show you're listening and engaged"""
        listening_markers = ['tell me more', 'i understand', 'i see', 'that makes sense',
                            'help me understand', 'i hear you', 'go on', 'continue',
                            'interesting', 'i appreciate']
        return sum(1 for marker in listening_markers if marker in text)
    
    def get_empathy_score(self):
        """Calculate overall empathy score (0-100)."""
        if not self.conversation_data:
            return 0
        
        # Component scores (how many markers were found)
        avg_sentiment = statistics.mean([d['sentiment'] for d in self.conversation_data])
        total_open_q = sum([d['open_questions'] for d in self.conversation_data])
        total_emotion = sum([d['emotion_words'] for d in self.conversation_data])
        total_perspective = sum([d['perspective_taking'] for d in self.conversation_data])
        total_listening = sum([d['active_listening'] for d in self.conversation_data])
        
        num_messages = len(self.conversation_data)
        
        # Normalize scores (0-20 points each dimension, total = 100)
        sentiment_score = (avg_sentiment + 1) * 10  # -1 to +1 → 0 to 20
        question_score = min(total_open_q / num_messages * 20, 20)
        emotion_score = min(total_emotion / num_messages * 10, 20)
        perspective_score = min(total_perspective / num_messages * 20, 20)
        listening_score = min(total_listening / num_messages * 20, 20)
        
        total = sentiment_score + question_score + emotion_score + perspective_score + listening_score
        
        return total  # Score out of 100
    
    def get_report(self):
        """Generate detailed empathy report with interpretation and recommendations."""
        score = self.get_empathy_score()
        
        # Interpret the score
        interpretation = ""
        if score >= 80:
            interpretation = "Excellent - Strong empathy demonstrated consistently"
        elif score >= 60:
            interpretation = "Good - Regular empathic responses with room to grow"
        elif score >= 40:
            interpretation = "Moderate - Shows awareness but inconsistent"
        elif score >= 20:
            interpretation = "Developing - Building empathy skills"
        else:
            interpretation = "Needs Practice - Focus on active listening"
        
        # Calculate metrics
        avg_sentiment = statistics.mean([d['sentiment'] for d in self.conversation_data]) if self.conversation_data else 0
        total_open_q = sum([d['open_questions'] for d in self.conversation_data])
        total_emotion = sum([d['emotion_words'] for d in self.conversation_data])
        total_perspective = sum([d['perspective_taking'] for d in self.conversation_data])
        total_listening = sum([d['active_listening'] for d in self.conversation_data])
        
        # Generate report
        report = f"""
╔═══════════════════════════════════════════════════════════╗
║           EMPATHY TRAINING ANALYSIS REPORT                ║
╚═══════════════════════════════════════════════════════════╝

📊 OVERALL EMPATHY SCORE: {score:.1f}/100
   {interpretation}

📈 CONVERSATION METRICS:
   • Total Messages: {len(self.conversation_data)}
   • Average Sentiment: {avg_sentiment:.2f} (-1 to +1)
   • Total Open Questions: {total_open_q}
   • Total Emotion Words: {total_emotion}
   • Perspective-Taking Attempts: {total_perspective}
   • Active Listening Markers: {total_listening}

💡 RECOMMENDATIONS:
"""
        # Add specific suggestions based on weaknesses
        if total_open_q < len(self.conversation_data):
            report += "   • Try asking more open-ended questions (what/how/why)\n"
        if total_emotion < len(self.conversation_data) * 2:
            report += "   • Acknowledge emotions more explicitly (\"you seem...\", \"you feel...\")\n"
        if total_perspective < len(self.conversation_data):
            report += "   • Practice perspective-taking (\"from your view...\", \"in your situation...\")\n"
        if avg_sentiment < 0.2:
            report += "   • Use warmer, more supportive language\n"
        if total_listening < len(self.conversation_data):
            report += "   • Show more active listening (\"tell me more\", \"I understand\")\n"
        
        if score >= 80:
            report += "   • Excellent work! You're demonstrating strong empathy consistently.\n"
        
        return report

# Initialize analyzer
empathy_analyzer = EmpathyAnalyzer()

print("✅ Empathy Analysis Module loaded!")
print("📊 Student responses will be analyzed for empathy indicators")
print("⏱️  After 20 messages, you'll receive a comprehensive empathy report")

In [ ]:
# Global variable to store which PDF files were used for the last answer
last_sources_used = []

def retrieve_relevant_context(query, n_results=NUM_RETRIEVED_DOCS):
    """Search the database for relevant document chunks and return with metadata."""
    global last_sources_used
    try:
        # Convert question to numbers (embedding) for searching
        query_embedding = embedding_model.encode([query])
        
        # Search ChromaDB for similar chunks
        results = collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=min(n_results, collection.count())  # Get up to N chunks
        )
        
        # Extract documents and metadata
        documents = results['documents'][0] if results['documents'] else []
        metadatas = results['metadatas'][0] if results['metadatas'] else []
        
        # Track which PDF files were used (for source citations)
        last_sources_used = []
        if metadatas:
            seen_sources = set()  # Avoid duplicates
            for meta in metadatas[:2]:  # Only track the 2 chunks we actually use
                source_name = meta.get('source', 'Unknown')
                if source_name not in seen_sources:
                    last_sources_used.append(source_name)
                    seen_sources.add(source_name)
        
        return documents
    except Exception as e:
        print(f"Error in retrieval: {str(e)}")
        last_sources_used = []
        return []

def generate_response_sync(question):
    """Call Gemini API to generate an answer (synchronous version).
    
    This runs in a separate thread with timeout control.
    """
    # Step 1: Find relevant chunks from PDFs
    context_docs = retrieve_relevant_context(question)
    
    # Step 2: Reduce context size for faster processing
    if context_docs:
        context_docs = context_docs[:2]  # Use only 2 chunks (not all 7)
        context = "\n\n".join(context_docs)
        context = context[:1500]  # Limit to 1500 characters
    else:
        context = "No relevant documents found."
    
    # Step 3: Create prompt with context + persona + question
    prompt = f"""{PERSONA_DESCRIPTION}

Context: {context}

Question: {question}

Answer in your persona's style:"""
    
    # Step 4: Call Gemini API
    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=TEMPERATURE,  # Creativity level
            max_output_tokens=200,  # Response length (reduced for speed)
            top_p=0.95,  # Sampling diversity
            top_k=40,  # Sampling diversity
        ),
    )
    
    return response.text

async def generate_response_async(question, chat_history=None, timeout_seconds=30):
    """Async wrapper with 30-second timeout to prevent hanging.
    
    Args:
        question: The user's question
        chat_history: Previous conversation (optional)
        timeout_seconds: Maximum wait time (default: 30 seconds)
    
    Returns:
        The chatbot's response or error message
    """
    try:
        # Run the synchronous function in a separate thread with timeout
        response_text = await asyncio.wait_for(
            asyncio.to_thread(generate_response_sync, question),
            timeout=timeout_seconds  # Hard timeout after 30 seconds
        )
        
        return response_text
    
    except asyncio.TimeoutError:
        # API took too long
        return "⏱️ **Timeout Error**\n\nThe API took too long (>30 seconds).\n\n**Try:**\n1. Ask a shorter, simpler question\n2. Wait 30 seconds and try again\n3. Check your internet connection"
    
    except Exception as e:
        error_str = str(e).lower()
        
        # Rate limit errors (too many requests)
        if "429" in str(e) or "quota" in error_str or "rate limit" in error_str or "resource exhausted" in error_str:
            return "⚠️ **Rate Limit**\n\nThe free API has request limits. Please wait 1-2 minutes and try again."
        
        # Connection/timeout errors
        elif "timeout" in error_str or "connection" in error_str or "deadline" in error_str:
            return "⚠️ **Connection Error**\n\nAPI connection failed.\n\n**Try:**\n1. Check your internet connection\n2. Wait a moment and try again\n3. Check API status at https://status.cloud.google.com/"
        
        # Content safety filter
        elif "blocked" in error_str or "safety" in error_str:
            return "⚠️ **Content Blocked**\n\nResponse blocked by safety filters. Try rephrasing your question."
        
        # API key/auth errors
        elif "api" in error_str or "invalid" in error_str or "auth" in error_str or "key" in error_str:
            return f"⚠️ **API Error**\n\n{str(e)}\n\n**Check:**\n1. Your API key is valid\n2. You have internet connectivity"
        
        # Other errors
        else:
            return f"❌ **Unexpected Error**\n\n{str(e)[:200]}\n\n**Try:**\n1. Restarting the kernel (Runtime → Restart runtime)\n2. Re-running all cells"

print("✅ Async RAG function created with 30-second timeout!")
print("🤖 The chatbot will respond within 30 seconds or show a timeout message.")
print("⏱️  Expected response time: 5-15 seconds")
if SHOW_SOURCES:
    print("📚 Source citations enabled - you'll see which PDFs were used!")

In [ ]:
async def chat_interface(message, history):
    """Chat handler with empathy tracking.
    
    Args:
        message: Current user message
        history: List of previous messages
    
    Returns:
        Bot's response (includes empathy report after 20 messages)
    """
    # Analyze student's message for empathy
    empathy_analysis = empathy_analyzer.analyze_message(message)
    
    # Generate bot response
    response = await generate_response_async(message, history)
    
    # Append source citations if enabled (before empathy progress)
    if SHOW_SOURCES and last_sources_used:
        response += "\n\n---\n**📚 Sources used:**\n"
        for i, source in enumerate(last_sources_used, 1):
            response += f"{i}. {source}\n"
        response += "\n💡 *Set `SHOW_SOURCES = False` in Cell 8 to hide citations*"
    
    # Check if we've reached 20 messages
    message_count = len(empathy_analyzer.conversation_data)
    
    if message_count == 20:
        # Generate and append empathy report
        report = empathy_analyzer.get_report()
        response += f"\n\n{'='*60}\n{report}\n{'='*60}"
        response += "\n\n🎉 **Empathy training session complete!** You can continue chatting if you'd like."
    elif message_count < 20:
        # Show progress
        remaining = 20 - message_count
        response += f"\n\n📊 *Message {message_count}/20 • {remaining} messages until empathy report*"
    
    return response

# Create Gradio ChatInterface with empathy tracking
demo = gr.ChatInterface(
    fn=chat_interface,  # Function to handle chat
    title=f"🤖 Chat with {PERSONA_NAME} - Empathy Training Edition",
    description=f"""Ask questions and get answers in {PERSONA_NAME}'s distinctive style!
    
    📚 **Documents:** Your uploaded PDF files will be used to provide factual answers.
    
    📊 **Empathy Tracking:** Your messages are analyzed for empathy. After 20 messages, you'll receive a detailed report with your score and improvement tips.
    
    💡 **Tips:** 
    - Ask questions related to your PDF documents
    - Demonstrate empathy: use "you feel", ask "how/why", acknowledge emotions
    {'- Source citations are ON - see which documents were used!' if SHOW_SOURCES else ''}
    
    ⏱️ **Response time:** 5-15 seconds (30 second max)
    """,
    examples=[
        "What are your main beliefs or values?",
        "How did that experience make you feel?",
        "Tell me more about your perspective on this topic.",
        "You seem passionate about this - what drives that feeling?",
        "From your point of view, what are your greatest achievements?",
    ],
)

# Launch with stable external link
print("=" * 80)
print("🚀 LAUNCHING EMPATHY TRAINING CHATBOT")
print("=" * 80)
print("\n⚠️  IMPORTANT: Use the PUBLIC LINK below (not the Colab interface)\n")
print("The public link is stable. The Colab interface may restart.\n")
if SHOW_SOURCES:
    print("📚 Source citations are ON (change SHOW_SOURCES in Cell 8 to disable)\n")
print("👇 COPY AND OPEN THIS LINK:\n")

demo.launch(
    share=True,      # Create public link
    inline=False,    # Don't use Colab interface (unstable)
    debug=True       # Show errors
)

print("\n" + "=" * 80)
print("✅ Chatbot is live with empathy tracking!")
print("=" * 80)
print("\n📋 INSTRUCTIONS:")
print("   1. Find the 'Running on public URL:' message above")
print("   2. Copy the https://xxxxx.gradio.live URL")
print("   3. Open it in a new browser tab")
print("   4. Start chatting!")
print("   5. After 20 messages, you'll get your empathy report")
if SHOW_SOURCES:
    print("   6. Check the bottom of each answer for source citations")
print("\n💡 Link expires after 72 hours of inactivity.\n")

---
## Step 9: Export Conversation Data (Optional)

After completing your conversation, you can export your empathy data to CSV for further analysis or to track your progress over time.

**To export:**
1. Uncomment the last line in the cell below
2. Run the cell
3. Find the file in your Google Drive root folder
4. Download it for offline analysis

In [ ]:
import csv
import os

def export_conversation_to_csv(filename='empathy_conversation.csv'):
    """Export conversation data with empathy metrics to CSV."""
    
    if len(empathy_analyzer.conversation_data) == 0:
        print("❌ No conversation data to export yet!")
        print("💬 Start chatting first, then come back to export.")
        return
    
    # Create filepath in Google Drive
    filepath = f"/content/drive/MyDrive/{filename}"
    
    # Write to CSV
    with open(filepath, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['timestamp', 'message', 'word_count', 
                                                'sentiment', 'open_questions', 'emotion_words',
                                                'perspective_taking', 'active_listening'])
        writer.writeheader()
        writer.writerows(empathy_analyzer.conversation_data)
    
    # Calculate and print summary
    score = empathy_analyzer.get_empathy_score()
    
    print(f"✅ Conversation exported successfully!")
    print(f"📁 File location: {filepath}")
    print(f"📊 Total messages: {len(empathy_analyzer.conversation_data)}")
    print(f"🎯 Your empathy score: {score:.1f}/100")
    print("\n💡 You can download this file from Google Drive to analyze further!")

# Uncomment the line below to export your conversation:
# export_conversation_to_csv()


---
## 🔧 Troubleshooting

### API Key Issues:
- **Error: "Invalid API key"**
  - Get a new key from: https://aistudio.google.com/app/apikey
  - Make sure you copied the entire key
  - Replace `YOUR_API_KEY_HERE` in Step 4

### PDF Issues:
- **"File not found" errors:**
  - Check that Google Drive is mounted (Step 3)
  - Verify PDF file paths are correct
  - Make sure paths start with `/content/drive/MyDrive/`
  
- **"No text extracted":**
  - PDF might be scanned images (not searchable text)
  - PDF might be password-protected
  - Try opening the PDF to verify it has selectable text

### Response Issues:
- **Responses don't match persona:**
  - Make `PERSONA_DESCRIPTION` more detailed and specific
  - Add more example phrases/words they use
  
- **Responses aren't relevant:**
  - Increase `NUM_RETRIEVED_DOCS` (try 5 or 7)
  - Make sure PDFs contain information about the topic
  - Ask more specific questions

### Performance Issues:
- **Colab disconnects or times out:**
  - This is normal for free Colab after ~12 hours
  - Save your work and restart
  - Keep the browser tab active

### Need More Help?
- Check Google Gemini API status: https://status.cloud.google.com/
- Verify free tier limits haven't been exceeded

---
## 🎓 Understanding the Technology

### What is RAG (Retrieval-Augmented Generation)?
RAG combines two technologies:
1. **Retrieval**: Searching documents for relevant information
2. **Generation**: Using AI to create natural responses

### How This Notebook Works:
1. **PDFs → Text**: Extract text from your PDF files
2. **Text → Chunks**: Split into smaller, searchable pieces
3. **Chunks → Embeddings**: Convert to numerical representations
4. **Store in Database**: Save in ChromaDB for fast searching
5. **User Asks Question**: You type a question
6. **Search Database**: Find most relevant chunks
7. **AI Generates Answer**: Gemini creates response using context
8. **Apply Persona**: Format response in persona's style

### Why This Approach?
- ✅ **Accurate**: Responses based on your actual documents
- ✅ **Up-to-date**: Use any current information in PDFs
- ✅ **Customizable**: Change persona, style, and content easily
- ✅ **Free**: No paid services required
- ✅ **Educational**: Students learn modern AI techniques

### Technologies Used:
- **Google Gemini**: Free AI language model
- **ChromaDB**: Vector database for semantic search
- **Sentence Transformers**: Convert text to embeddings
- **Gradio**: Create chat interface
- **PyPDF**: Read PDF files

---
## 🚀 Next Steps & Future Features

### Ideas for Enhancement:
1. **Add more document types**: Support Word docs, web pages, etc.
2. **Conversation memory**: Remember previous questions in the chat
3. **Source citations**: Show which PDF the answer came from
4. **Multiple personas**: Switch between different personalities
5. **Voice input/output**: Add speech recognition and text-to-speech
6. **Fact-checking mode**: Verify claims against documents
7. **Export conversations**: Save chat history
8. **Advanced search**: Filter by document, date, topic, etc.

### Learning Resources:
- Google Gemini API Docs: https://ai.google.dev/docs
- ChromaDB Documentation: https://docs.trychroma.com/
- Gradio Documentation: https://www.gradio.app/docs/
- RAG Overview: https://python.langchain.com/docs/use_cases/question_answering/

---

**Happy Chatting! 🎉**

*Created for educational purposes. Completely free and customizable.*